In [1]:
#@title 查看显卡
!nvidia-smi

Mon Feb 19 06:26:35 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
#@title 克隆代码仓库
!git clone -b v2.0.1 --single-branch https://github.com/cncbec/Bert-VITS2/

Cloning into 'Bert-VITS2'...
remote: Enumerating objects: 1754, done.
remote: Total 1754 (delta 0), reused 0 (delta 0), pack-reused 1754
Receiving objects: 100% (1754/1754), 8.12 MiB | 13.63 MiB/s, done.
Resolving deltas: 100% (1024/1024), done.


In [ ]:
#@title 安装所需要的依赖
%cd /content/Bert-VITS2/
!pip install -r requirements.txt

In [ ]:
#@title 通过直链下载必要的文件
!wget -P emotional/wav2vec2-large-robust-12-ft-emotion-msp-dim/ https://huggingface.co/audeering/wav2vec2-large-robust-12-ft-emotion-msp-dim/resolve/main/pytorch_model.bin
!wget -P bert/chinese-roberta-wwm-ext-large/ https://huggingface.co/hfl/chinese-roberta-wwm-ext-large/resolve/main/flax_model.msgpack
!wget -P bert/chinese-roberta-wwm-ext-large/ https://huggingface.co/hfl/chinese-roberta-wwm-ext-large/resolve/main/pytorch_model.bin
!wget -P bert/chinese-roberta-wwm-ext-large/ https://huggingface.co/hfl/chinese-roberta-wwm-ext-large/resolve/main/tf_model.h5
!wget -P bert/bert-base-japanese-v3/ https://huggingface.co/cl-tohoku/bert-base-japanese-v3/resolve/main/flax_model.msgpack
!wget -P bert/bert-base-japanese-v3/ https://huggingface.co/cl-tohoku/bert-base-japanese-v3/resolve/main/pytorch_model.bin
!wget -P bert/bert-base-japanese-v3/ https://huggingface.co/cl-tohoku/bert-base-japanese-v3/resolve/main/tf_model.h5
!wget -P bert/bert-base-japanese-v3/ https://huggingface.co/cl-tohoku/bert-base-japanese-v3/resolve/main/tokenizer_config.json
!wget -P bert/deberta-v3-large/ https://huggingface.co/microsoft/deberta-v3-large/resolve/main/pytorch_model.bin
!wget -P bert/deberta-v3-large/ https://huggingface.co/microsoft/deberta-v3-large/resolve/main/pytorch_model.generator.bin
!wget -P bert/deberta-v3-large/ https://huggingface.co/ku-nlp/deberta-v2-large-japanese/resolve/main/spm.model
!wget -P bert/deberta-v3-large/ https://huggingface.co/microsoft/deberta-v3-large/resolve/main/tf_model.h5
!wget -P bert/deberta-v2-large-japanese/ https://huggingface.co/ku-nlp/deberta-v2-large-japanese/resolve/main/model.safetensors
!wget -P bert/deberta-v2-large-japanese/ https://huggingface.co/ku-nlp/deberta-v2-large-japanese/resolve/main/pytorch_model.bin
!wget -P bert/deberta-v2-large-japanese/ https://huggingface.co/ku-nlp/deberta-v2-large-japanese/resolve/main/spm.model

In [ ]:
#@title 下载底模文件
#@markdown 数据集名称（**人物的英文/拼音名**，与建数据文件夹时统一；不带zip。）
DATASETNAME = "tongyong"  #@param {type:"string"}
!wget -P {DATASETNAME}/models/ https://huggingface.co/Mengxing56546/Bert-VITS2_v2.0.1_base_models/resolve/main/DUR_0.pth
!wget -P {DATASETNAME}/models/ https://huggingface.co/Mengxing56546/Bert-VITS2_v2.0.1_base_models/resolve/main/D_0.pth
!wget -P {DATASETNAME}/models/ https://huggingface.co/Mengxing56546/Bert-VITS2_v2.0.1_base_models/resolve/main/G_0.pth


In [6]:
#@title 挂载谷歌云盘
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title (有数据集文件)从谷歌云盘加载打包好的数据集进行预处理
#@markdown 压缩包路径（谷歌盘路径，传到dataset的就不改这个，没有dataset文件夹就新建一个）
ZIP_PATH = "/content/drive/MyDrive/dataset/"  #@param {type:"string"}
ZIP_NAME = ZIP_PATH + DATASETNAME
!unzip -d /content/Bert-VITS2/raw {ZIP_NAME}.zip

In [ ]:
#@title (有数据集文件)从谷歌云盘添加标注文件到实例
%cd /content
!cp /content/drive/MyDrive/dataset/clean_barbara.list Bert-VITS2/filelists
%cd ./Bert-VITS2/filelists
!mv clean_barbara.list {DATASETNAME}.list

/content
cp: cannot stat '/content/drive/MyDrive/dataset/clean_barbara.list': No such file or directory
/content/Bert-VITS2/filelists
mv: cannot stat 'clean_barbara.list': No such file or directory


In [ ]:
#@title (没有数据集文件)把长语音以人名（英文）命名。放入data文件夹下
#比如：data/gaoxiangdong.wav

In [ ]:
!pip install -U openai-whisper

In [ ]:
#@title (没有数据集文件)切分长语音，并添加备注文件
!python clean_list.py

Detected language: zh
电影频道的观众朋友们大家好我是赵丽颖祝大家春节快乐
Detected language: zh
龙年大吉对,对自己来说是一个突破然后也是一个挑战,也是一个审视吧
Detected language: zh
就是在跟导演在梳理角色的过程当中然后就是产生的一些想法一些可行一些能够让角色
Detected language: zh
能够更准确的表达他在戏当中的作用
Detected language: zh
會想到的一個想法很對我們一下子所有的
Detected language: zh
逻辑所有的人物关系然后所有的
Detected language: zh
就是這個
Detected language: zh
就都想通了对它就是会是一个
Detected language: zh
怎么说,他不能作为全部,但是他是作为一个角色的设定来说是一个很好的
Detected language: zh
或者是对于我
Detected language: zh
它會隨著我年紀大 增長然後慢慢這顆流星就過了


In [14]:
#@title (没有数据集文件)生成对应人名的list文件
#生成filelists下的通用list文件
!sed -i 's/tongyong/{DATASETNAME}/g' slice.py
!python slice.py

In [15]:
#@title 重采样
%cd /content/Bert-VITS2
!python resample.py
!python resample.py

/content/Bert-VITS2
已根据默认配置文件default_config.yml生成配置文件config.yml。请按该配置文件的说明进行配置后重新运行。
如无特殊需求，请勿修改default_config.yml或备份该文件。
40it [00:00, 86.77it/s]
音频重采样完毕!


In [21]:
#@title 执行preprocess_text.py文件，划分训练集和验证集
#处理完成后，如果train.list中的数量没有，可以手动从val.list剪切过去部分。
%cd /content/Bert-VITS2
!sed -i 's/datasetname/{DATASETNAME}/g' config.py
!python preprocess_text.py

/content/Bert-VITS2
  0% 0/40 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.652 seconds.
Prefix dict has been built successfully.
100% 40/40 [00:00<00:00, 44.29it/s]
总重复音频数：0，总未找到的音频数:0
['./dataset/zhaoliying/zhaoliying_6.wav|zhaoliying|ZH|就是這個|_ j iu sh ir zh e g e _|0 4 4 4 4 4 4 4 4 0|1 2 2 2 2 1\n', './dataset/zhaoliying/zhaoliying_7.wav|zhaoliying|ZH|就都想通了对它就是会是一个|_ j iu d ou x iang t ong l e d ui t a j iu sh ir h ui sh ir y i g e _|0 4 4 1 1 3 3 1 1 5 5 4 4 1 1 4 4 4 4 4 4 4 4 2 2 5 5 0|1 2 2 2 2 2 2 2 2 2 2 2 2 2 1\n', './dataset/zhaoliying/zhaoliying_9.wav|zhaoliying|ZH|或者是对于我|_ h uo zh e sh ir d ui y v w o _|0 4 4 3 3 4 4 4 4 2 2 3 3 0|1 2 2 2 2 2 2 1\n', './dataset/zhaoliying/zhaoliying_8.wav|zhaoliying|ZH|怎么说,他不能作为全部,但是他是作为一个角色的设定来说是一个很好的|_ z en m e sh uo , t a b u n eng z uo w ei q van b u , d an sh ir t a sh ir z uo w ei y i g e j ve s e d e sh e d ing l ai sh uo sh ir y i g e h en h ao d

In [17]:
#@title 执行bert_gen.py文件，生成.pt文件
!python bert_gen.py

  0% 0/40 [00:00<?, ?it/s]Some weights of the model checkpoint at ./bert/chinese-roberta-wwm-ext-large were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at ./bert/chinese-roberta-wwm-ext-large were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you a

In [ ]:
#@title 执行emo_gen.py文件
!python emo_gen.py


In [ ]:
#@title 已经预处理过数据集的话，就可以跳过预处理部分 直接从云盘解压处理过的数据 以及配置文件
#@markdown 从谷歌云盘加载预处理过的数据集，文件夹名和你备份的时候输入的一样
back_up_name = "tongyong"  #@param {type:"string"} 同下述：dataset_name_drive
BACK_UP_DATASET_PATH = "/content/drive/MyDrive/dataset/" + back_up_name

!unzip {BACK_UP_DATASET_PATH}/dataset.zip -d /
!cp {BACK_UP_DATASET_PATH}/val.list filelists/val.list
!cp {BACK_UP_DATASET_PATH}/train.list filelists/train.list
!cp {BACK_UP_DATASET_PATH}/config.json configs/config.json

Archive:  /content/drive/MyDrive/dataset/ying1/ying.zip
   creating: /content/Bert-VITS2/ying/
   creating: /content/Bert-VITS2/ying/models/
  inflating: /content/Bert-VITS2/ying/models/G_2000.pth  
  inflating: /content/Bert-VITS2/ying/models/DUR_0.pth  
  inflating: /content/Bert-VITS2/ying/models/DUR_1000.pth  
  inflating: /content/Bert-VITS2/ying/models/githash  
   creating: /content/Bert-VITS2/ying/models/.ipynb_checkpoints/
  inflating: /content/Bert-VITS2/ying/models/DUR_2000.pth  
   creating: /content/Bert-VITS2/ying/models/eval/
  inflating: /content/Bert-VITS2/ying/models/eval/events.out.tfevents.1700927859.0f826d94c242.5838.1  
  inflating: /content/Bert-VITS2/ying/models/D_2000.pth  
  inflating: /content/Bert-VITS2/ying/models/train.log  
  inflating: /content/Bert-VITS2/ying/models/events.out.tfevents.1700927859.0f826d94c242.5838.0  


In [ ]:
#@title 开始训练
#@markdown 数据集名称（**人物的英文/拼音名**，与建数据文件夹时统一；不带zip。）
DATASETNAME = "tongyong"  #@param {type:"string"}
!python train_ms.py -m {DATASETNAME} -c configs/config.json

In [ ]:
#@title 将数据集和相关文件保存到谷歌云盘的dataset文件夹下，方便下次训练使用
#压缩dataset和logs文件夹
!zip -r dataset.zip /content/Bert-VITS2/dataset
!zip -r {DATASETNAME}.zip /content/Bert-VITS2/{DATASETNAME}
#@markdown 自定义备份到谷歌云盘的dataset文件夹下的数据集文件夹名，避免混淆
dataset_name_drive = "tongyong"  #@param {type:"string"} 同上述：back_up_name
DATASET_PATH_DRIVE = "/content/drive/MyDrive/dataset/" + dataset_name_drive
!mkdir -p {DATASET_PATH_DRIVE}

!cp /content/Bert-VITS2/dataset.zip "{DATASET_PATH_DRIVE}"
!cp /content/Bert-VITS2/{DATASETNAME}.zip "{DATASET_PATH_DRIVE}"
!cp configs/config.json "{DATASET_PATH_DRIVE}"
!cp filelists/train.list "{DATASET_PATH_DRIVE}"
!cp filelists/val.list "{DATASET_PATH_DRIVE}"

In [ ]:
#!pip install wavfile

In [ ]:
#!cp -r /content/drive/MyDrive/BertVits2_EasyWav2lip_Fasefusion/gxd /content/Bert-VITS2/gxd

In [ ]:
#!cp /content/drive/MyDrive/BertVits2_EasyWav2lip_Fasefusion/source/config.json /content/Bert-VITS2/configs

In [ ]:
#@title 推理
#@markdown 模型名称
Model_File = "G_850"  #@param {type:"string"}
!sed -i 's/Model_File/{Model_File}/g' config.py
!sed -i 's/datasetname/{DATASETNAME}/g' config.py
#!python webui.py
!python inference_single.py

| numexpr.utils | INFO | NumExpr defaulting to 2 threads.
/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
| utils | INFO | Loaded checkpoint './gxd/models/G_4000.pth' (iteration 401)
| langid.langid | INFO | initializing identifier
[['为什么反者道之动？什么叫反者道之动呢？', 'I will to tell you!', '就世界上所有的事物都必然走向他自己的反面,由于这一点,让中国中华民族有一种智慧.']] [['ZH', 'EN', 'ZH']]
Building prefix dict from the default dictionary ...
| jieba | DEBUG | Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
| jieba | DEBUG | Loading model from cache /tmp/jieba.cache
Loading model cost 0.620 seconds.
| jieba | DEBUG | Loading model cost 0.620 seconds.
Prefix dict has been built successfully.
| jieba | DEBUG | Prefix dict has been built su

In [26]:
!python inference_single.py --model tongyong/models/G_1200.pth --speaker xiaozhan --text 赵石蕾，今天是你的生日，感谢你为孩子、为爱人、为国家做出这么多的自我牺牲，愿你的生日充满无穷的快乐，愿你今天的回忆温馨，愿你所有的梦想甜蜜，愿你这一年称心如意！生日快乐！

| numexpr.utils | INFO | NumExpr defaulting to 2 threads.
| httpx | INFO | HTTP Request: GET https://api.gradio.app/gradio-messaging/en "HTTP/1.1 200 OK"
/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
| utils | INFO | Loaded checkpoint 'tongyong/models/G_1200.pth' (iteration 151)
| langid.langid | INFO | initializing identifier
[['赵石蕾，今天是你的生日，感谢你为孩子、为爱人、为国家做出这么多的自我牺牲，愿你的生日充满无穷的快乐，愿你今天的回忆温馨，愿你所有的梦想甜蜜，愿你这一年称心如意！生日快乐！']] [['ZH']]
Building prefix dict from the default dictionary ...
| jieba | DEBUG | Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
| jieba | DEBUG | Loading model from cache /tmp/jieba.cache
Loading model cost 0.639 seconds.
| jieba | DEBUG | Loading model cost 0.639 seconds.
Pr